# ・各センサーの状態取得

In [2]:
import rospy
from khi_robot_msgs.srv import KhiRobotCmd
rospy.init_node("driver_status_call_node")

### 1) サービスファイルの呼び出し定義
サービスファイルの仕様<br>
string type<br>
string cmd<br>
---<br>
int32 driver_ret<br>
int32 as_ret<br>
string cmd_ret</p>

In [3]:
service = '/khi_robot_command_service'

### 2) Fシリーズコントローラ(30F60F-AC01)にコマンド送信を行う関数

In [4]:
def cmdhandler_client(type_arg , cmd_arg) -> dict:
    """
        cmdhandler_client
            - type_arg: str = 'driver' | 'as'
            - cmd_arg: str = 'command'
    """
    rospy.wait_for_service(service)
    try:
        khi_robot_command_service = rospy.ServiceProxy(service, KhiRobotCmd)
        resp1 = khi_robot_command_service(type_arg, cmd_arg)
        return {"driver_ret": resp1.driver_ret, "as_ret": resp1.as_ret, "cmd_ret": resp1.cmd_ret}
    except rospy.ServiceException as e:
        rospy.loginfo('Service call failed: %s', e)

### 3) Fシリーズコントローラ(30F60F-AC01)の状態の取得　!以降からFシリーズコントローラ(30F60F-AC01)をドライバと呼ぶ!

In [10]:
ret = cmdhandler_client("driver", "get_status")
print(ret)

{'driver_ret': 0, 'as_ret': 0, 'cmd_ret': 'ACTIVE'}


### 4) ドライバのリセット

In [12]:
ret = cmdhandler_client("driver", "restart")
print(ret)

{'driver_ret': 0, 'as_ret': 0, 'cmd_ret': 'NOT INACTIVE/HOLDED/ERROR STATE'}


### 5) ドライバの状態確認

In [13]:
ret = cmdhandler_client("driver", "get_status")
print(ret)

{'driver_ret': 0, 'as_ret': 0, 'cmd_ret': 'ACTIVE'}


### 6) 収納レーンのソレノイド装置のオンオフ

In [14]:
# ソレノイドの状態を示すシグナル値を指定して状態を取得
result = cmdhandler_client('driver', 'get_signal 12')
print(result) # cmd_ret: 0 でソレノイドが作動していない状態

{'driver_ret': 0, 'as_ret': 0, 'cmd_ret': '0'}

In [18]:
# ソレノイドを起動 
result = cmdhandler_client('driver', 'set_signal 12') #　set_signalで起動
print(result)

{'driver_ret': 0, 'as_ret': 0, 'cmd_ret': ''}

In [19]:
# ソレノイドの起動停止 
result = cmdhandler_client('driver', 'set_signal -12') #　マイナス値の指定で起動停止
print(result)

{'driver_ret': 0, 'as_ret': 0, 'cmd_ret': ''}

### 7) 収納レーン 在荷センサー状態取得

In [25]:
# 空の状態
result = cmdhandler_client('driver', 'get_signal 1005')
print(result)

{'driver_ret': 0, 'as_ret': 0, 'cmd_ret': '0'}

In [26]:
# ワークが存在したときの状態
result = cmdhandler_client('driver', 'get_signal 1005')
print(result)

{'driver_ret': 0, 'as_ret': 0, 'cmd_ret': '-1'}

### 8) カラーセンサー台の在荷センサー・カラーセンサーの状態取得

In [28]:
# カラーセンサ台に何も置いてない状態
result = cmdhandler_client('driver', 'get_signal 1009')
print(result)

{'driver_ret': 0, 'as_ret': 0, 'cmd_ret': '0'}

In [29]:
# 置いている状態
result = cmdhandler_client('driver', 'get_signal 1009')
print(result)

{'driver_ret': 0, 'as_ret': 0, 'cmd_ret': '-1'}

### 色判別

In [33]:
# ワークの色が青のとき
result = ~ int(cmdhandler_client('driver', 'get_signal 1009')['cmd_ret'])
result1 = ~ int(cmdhandler_client('driver', 'get_signal 1010')['cmd_ret'])
result2 = ~ int(cmdhandler_client('driver', 'get_signal 1011')['cmd_ret'])
result3 = ~ int(cmdhandler_client('driver', 'get_signal 1012')['cmd_ret']) 
print(result, result1, result2, result3)

0 -1 -1 -1


In [34]:
# ワークの色が赤のとき
result = ~ int(cmdhandler_client('driver', 'get_signal 1009')['cmd_ret'])
result1 = ~ int(cmdhandler_client('driver', 'get_signal 1010')['cmd_ret'])
result2 = ~ int(cmdhandler_client('driver', 'get_signal 1011')['cmd_ret'])
result3 = ~ int(cmdhandler_client('driver', 'get_signal 1012')['cmd_ret']) 
print(result, result1, result2, result3)

0 0 -1 -1


In [35]:
# ワークの色が黃のとき
result = ~ int(cmdhandler_client('driver', 'get_signal 1009')['cmd_ret'])
result1 = ~ int(cmdhandler_client('driver', 'get_signal 1010')['cmd_ret'])
result2 = ~ int(cmdhandler_client('driver', 'get_signal 1011')['cmd_ret'])
result3 = ~ int(cmdhandler_client('driver', 'get_signal 1012')['cmd_ret']) 
print(result, result1, result2, result3)

0 0 0 -1


In [36]:
# ワークの色がアルミのとき
result = ~ int(cmdhandler_client('driver', 'get_signal 1009')['cmd_ret'])
result1 = ~ int(cmdhandler_client('driver', 'get_signal 1010')['cmd_ret'])
result2 = ~ int(cmdhandler_client('driver', 'get_signal 1011')['cmd_ret'])
result3 = ~ int(cmdhandler_client('driver', 'get_signal 1012')['cmd_ret']) 
print(result, result1, result2, result3)

0 0 0 0


### 9) ロボットハンドの開閉

In [78]:
# 状態のリセット
cmdhandler_client('driver', 'set_signal -17')
cmdhandler_client('driver', 'set_signal -18')
result = cmdhandler_client('driver', 'get_signal 1018')
print(result)

{'driver_ret': 0, 'as_ret': 0, 'cmd_ret': '0'}


In [86]:
# アームを開く
result_open_1 = cmdhandler_client('driver', 'set_signal 17')
result_open_2 = cmdhandler_client('driver', 'set_signal -17')
result = cmdhandler_client('driver', 'get_signal 1018')
print(result_open_1, result_open_2, result)

{'driver_ret': 0, 'as_ret': 0, 'cmd_ret': ''} {'driver_ret': 0, 'as_ret': 0, 'cmd_ret': ''} {'driver_ret': 0, 'as_ret': 0, 'cmd_ret': '0'}


In [87]:
# アームを閉じる
result_open_1 = cmdhandler_client('driver', 'set_signal 18')
result_open_2 = cmdhandler_client('driver', 'set_signal -18')
result = cmdhandler_client('driver', 'get_signal 1018')
print(result_open_1, result_open_2, result)

{'driver_ret': 0, 'as_ret': 0, 'cmd_ret': ''} {'driver_ret': 0, 'as_ret': 0, 'cmd_ret': ''} {'driver_ret': 0, 'as_ret': 0, 'cmd_ret': '0'}


# RS007N_COMMANDER 

In [1]:
import rs007n_commander

In [2]:
driver = rs007n_commander.DriverCommander()

In [3]:
ColorType = rs007n_commander.ColorSensorType

In [ ]:
class DriverCommander(object):
    def DriverRestart(self) -> bool: return DriverRestart() # コントローラの状態確認関数
    def SolenoidOn(self, *a, **kw) -> bool: return SolenoidOnOff(*a, **kw) # ソレノイドのON/OFF関数 引数にON/OFFを行う秒数を指定デフォルとで２秒
    def HandOpen(self) -> bool: return HandOpen() # ロボットハンド関数 開
    def HandCloses(self) -> bool: return HandClose() # ロボットハンド関数 閉
    def TPSupply(self) -> bool: return TPSupply() # 収納レーンの在荷センサーの状態確認関数
    def EColorSensor(self) -> bool: return EColorSensor() # カラーセンサー台の在荷センサーの状態確認関数
    def ColorSensor(self) -> int: return ColorSensor # カラーセンサーの判別関数

In [ ]:
ColorTypeResult = driver.ColorSensor()

if ColorTypeResult == ColorType.RED:
    print("RED")
if ColorTypeResult == ColorType.BLUE:
    print("BLUE")
if ColorTypeResult == ColorType.YELLOW:
    print("YELLOW")
if ColorTypeResult == ColorType.ALUMINUM:
    print("ALUMINUM")
if ColorTypeResult == ColorType.UNKNOWN:
    print("UNKNOWN")